Задача заключается в проверке метода Chain of Thoughts и его улучшения на модели BLOOM 176B, для вычислений в Colab использовал библиотеку petals. Из-за очень медленной работы модели проводил эксперименты на очень ограниченной выборке (отсутствие эксперимента на цельной большой выборке компенсировалось результатами на случайных коротких выборках), 4 из 7 дней работы модель была перегружена, аккаунт Kaggle с малым количеством ресурсов не может поддерживать работу даже одного из 72 блоков, поэтому объединить несколько таких не получилось, а легкие BLOOM не показывают такого эффекта.


In [ ]:
!pip install petals sentence-transformers faiss-cpu gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 125.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 120.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 

In [ ]:
!wget https://raw.githubusercontent.com/openai/grade-school-math/master/grade_school_math/data/train.jsonl
!wget https://raw.githubusercontent.com/openai/grade-school-math/master/grade_school_math/data/test.jsonl

--2023-02-19 20:55:24--  https://raw.githubusercontent.com/openai/grade-school-math/master/grade_school_math/data/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4166206 (4.0M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   3.97M  --.-KB/s    in 0.01s   

2023-02-19 20:55:25 (292 MB/s) - ‘train.jsonl’ saved [4166206/4166206]

--2023-02-19 20:55:25--  https://raw.githubusercontent.com/openai/grade-school-math/master/grade_school_math/data/test.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
!gdown 1P7KGE64D-V2BSZiI6p8dJKfYZ4kfEzc3

Downloading...
From: https://drive.google.com/uc?id=1P7KGE64D-V2BSZiI6p8dJKfYZ4kfEzc3
To: /content/bloom_search_data.npy
100% 11.5M/11.5M [00:01<00:00, 8.21MB/s]


In [ ]:
import re
import torch
import faiss
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from petals import DistributedBloomForCausalLM
from transformers import BloomTokenizerFast 
from sklearn.metrics import accuracy_score

In [ ]:
def read_jsonl(path: str):
    with open(path) as f:
        return [json.loads(line) for line in f.readlines() if line]

In [ ]:
N = 20 # мало ресурсов
train = pd.DataFrame(read_jsonl('train.jsonl'))
test = pd.DataFrame(read_jsonl('test.jsonl'))
test = test.sample(N, random_state=42).reset_index(drop=True)

In [ ]:
embeddings = np.load('bloom_search_data.npy') # листайте до конца
train['embedding'] = list(embeddings)

In [ ]:
train.question[0]

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [ ]:
train.answer[0]

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

Разделяем датасет на вопрос/ответ/шаги. Содержит текстовое описание действий и формальное в виде выражений в <<>>. Выражения размечены очень плохо, встречаются <<40 = 40>> (пропущено начало)

In [ ]:
def get_formal_step(s):
    steps = re.findall(r'<<.*?>>', s.split('####')[0])
    return ' # '.join([f"({i}) {steps[i - 1].replace('>>', '').replace('<<', '')}" for i in range(1, len(steps) + 1)]) # читал, что нумерация улучшает генерацию
    

def get_text_step(s):
    return re.sub(r'<<.*?>>', '', s.split('####')[0]).replace('\n', ' ')

def get_answer(s):
    return s.split('####')[1].strip()

def clean_digit_comma(s):
    return re.sub('(?<=\d),(?=\d)', '', s) # чистит запятые в числительных 100,500

def clean_answer(s):
    return ''.join(x for x in s if x.isdigit())

In [ ]:
#train['answer'] = train.answer.apply(clean_digit_comma)
train['text_step'] = train.answer.apply(get_text_step)
train['formal_step'] = train.answer.apply(get_formal_step)
train['answer'] = train.answer.apply(get_answer)
#test['answer'] = test.answer.apply(clean_digit_comma)
test['text_step'] = test.answer.apply(get_text_step)
test['formal_step'] = test.answer.apply(get_formal_step)
test['answer'] = test.answer.apply(get_answer)
test['processed_answer'] = test.answer.apply(clean_answer)

In [ ]:
train.head(3)

,question,answer,embedding,text_step,formal_step
0,Natalia sold clips to 48 of her friends in Apr...,72,"[0.0161483, -0.029351955, 0.058499925, 0.00852...",Natalia sold 48/2 = 24 clips in May. Natalia s...,(1) 48/2=24 # (2) 48+24=72
1,Weng earns $12 an hour for babysitting. Yester...,10,"[0.025501177, 0.070720375, 0.04511817, 0.03589...",Weng earns 12/60 = $0.2 per minute. Working 50...,(1) 12/60=0.2 # (2) 0.2*50=10
2,Betty is saving money for a new wallet which c...,5,"[0.032309357, 0.14537759, -0.010018542, -0.044...","In the beginning, Betty has only 100 / 2 = $50...",(1) 100/2=50 # (2) 15*2=30 # (3) 100-50-30-15=5


### Baseline

Промт из статьи без указания действий

In [ ]:
arxiv_promt = '''Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: The answer is 6
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: The answer is 5
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: The answer is 39
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: The answer is 8
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: The answer is 9
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: The answer is 29
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: The answer is 33
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: The answer is 8
'''

Напишем первый датасет, который будет поддерживать использование статичного промта или num_samples примеров из обучающей выборки. Гипотеза состоит в том, что статичный промт не содержит примеров работы с долями, процентами, дробями, возможно, train set даст больше полезной информации 

In [ ]:
class MathDataset(Dataset):
    def __init__(self, df, examples, use_promt=None, num_samples=1):
        self.df = df
        self.examples = examples
        self.use_promt = use_promt
        self.num_samples = num_samples

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if self.use_promt:
            promt = self.use_promt
        else:
            example = self.examples.sample(self.num_samples)
            promt = ''
            for q, a in zip(example.question, example.answer):
                promt += 'Q: ' + q + '\n' + 'A: The answer is ' + a + '\n'
            
        sample = self.df.iloc[idx]
        promt += 'Q: ' + sample.question + '\nA: '
        return promt, sample.answer

In [ ]:
dataset = MathDataset(test, train, use_promt=arxiv_promt, num_samples=8) # промт из статьи
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: The answer is 6
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: The answer is 5
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: The answer is 39
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: The answer is 8
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: The answer is 9
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: The answer is 29
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, h

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "bigscience/bloom-petals"
tokenizer = BloomTokenizerFast.from_pretrained(model_name)
model = DistributedBloomForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.19G [00:00<?, ?B/s]

In [ ]:
def simple_inference(loader):
    preds = []
    for promt, answer in tqdm(loader):
        inputs = tokenizer(promt, return_tensors="pt", padding='longest')["input_ids"].to(device)
        outputs = model.generate(inputs, max_new_tokens = 128, eos_token_id=tokenizer.encode('\n')[0]) # \n - токен для окончания генерации
        preds.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    return preds

In [ ]:
preds = simple_inference(loader)

In [ ]:
print(preds[-1])

In [ ]:
def extract_answer(s):
    s = s.split('The answer is')[-1]
    return ''.join(x for x in s if x.isdigit())

In [ ]:
pred_answers = list(map(extract_answer, preds))
pred_answers[:10]

Формальный и нерепрезентативный подсчет метрики, стабильно не больше одного правильного ответа или 0.05, похоже, что модель дает случайный ответ

In [ ]:
accuracy_score(test.processed_answer, pred_answers)

In [ ]:
dataset = MathDataset(test, train, num_samples=8) # промт из train
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

In [ ]:
preds = simple_inference(loader)

In [ ]:
pred_answers = list(map(extract_answer, preds))
pred_answers[:10]

Ситуация не изменилась, здесь влияния промта нет

In [ ]:
accuracy_score(test.processed_answer, pred_answers)

### Chain of Thoughts

In [ ]:
arxiv_promt = '''Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8
'''

Возьмем полный промт из статьи, изменим датасет так, чтобы добавлял в промт последовательность действий разных типов

In [ ]:
class MathDataset(Dataset):
    def __init__(self, df, examples, use_promt=None, num_samples=1, step_col=''):
        self.df = df
        self.examples = examples
        self.num_samples = num_samples
        self.step_col = step_col
        self.use_promt = use_promt

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if self.use_promt:
            promt = self.use_promt
        else:
            example = self.examples.sample(self.num_samples)
            promt = ''
            for q, a, s in zip(example.question, example.answer, example[self.step_col]):
                if self.step_col:
                    promt += 'Q: ' + q + '\n' + 'A: ' + s + ' The answer is ' + a + '\n'
                else:
                    promt += 'Q: ' + q + '\n' + 'A: The answer is ' + a + '\n'
        sample = self.df.iloc[idx]
        
        promt += 'Q: ' + sample.question + '\nA: '
        return promt, sample.answer

In [ ]:
def cot_inference(loader):
    preds = []
    progress_bar = tqdm(loader)
    errors = 0

    for promt, answer in progress_bar:
        inputs = tokenizer(promt, return_tensors="pt", padding='longest')["input_ids"].to(device)
        outputs = model.generate(inputs, max_new_tokens = 128, eos_token_id=tokenizer.encode('\n')[0])
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded = [d[len(p):] for p, d in zip(promt, decoded)] # обрезаем промт в сгенерированных предложениях
        preds.extend(decoded)
        errors += sum(1 for d in decoded if 'The answer is' not in d) # сколько предложений не содержат ответа
        progress_bar.set_description(f'errors {errors}/{len(loader)}')
    return preds

In [ ]:
dataset = MathDataset(test, train, num_samples=8, step_col='formal_step')
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

Q: In the engineering department, 70% of the students are men and 180 are women. How many men are there?
A: (1) 6*70=420 The answer is 420
Q: A bottle can hold 2 cups of water. How many cups of water are needed to fill up 10 whole bottles and 5 half-capacity bottles?
A: (1) 10*2=20 # (2) 5*1=5 # (3) 25+5=30 The answer is 30
Q: Gail has some bills in her wallet which amount to $100. She has four $5 bills and three $20 bills, and the rest are $10 bills. How many $10 bills are in her wallet?
A: (1) 5*4=20 # (2) 20*3=60 # (3) 20/10=2 The answer is 2
Q: Misty's favorite number is 3 times smaller than Glory's favorite number. If Glory's favorite number is 450, what's the sum of their favorite numbers?
A: (1) 450/3=150 # (2) 150+450=600 The answer is 600
Q: A group of 300 athletes spent Saturday night at Ultimate Fitness Camp. The next morning, for 4 hours straight, they left the camp at a rate of 28 athletes per hour. Over the next 7 hours, a new group of athletes trickled into the camp at a

In [ ]:
preds = cot_inference(loader)

In [ ]:
def extract_answer(s):
    if 'The answer is' not in s:
        return np.nan
    s = s.split('The answer is')[-1].strip().split()[-1]
    return ''.join(x for x in s if x.isdigit())

In [ ]:
test['formal_preds'] = preds
test['formal_answer'] = preds
test['formal_answer'] = test['formal_answer'].apply(extract_answer)

In [ ]:
accuracy_score(test.processed_answer, test.formal_answer.fillna(''))

В среднем при таком подходе 20% ошибок, т.е. помимо неправильных ответов в 20% предсказаниях нет поля The answer is, а продолжается цепь мыслей, а точность в таком подходе стремится к 0

In [ ]:
def show_true(df, count=1, prefix=''):
    samples = df[df['processed_answer'] == df[prefix + '_answer']]
    if len(samples) == 0:
        return
    samples = samples.sample(count)
    for ts, fs, pr in zip(samples['text_step'], samples['formal_step'], samples[prefix + '_preds']):
        print('-' * 20)
        print('true step:')
        print(ts)
        print(fs)
        print()
        print('predict step:')
        print(pr)
        
def show_false(df, count=1, prefix=''):
    samples = df[df['processed_answer'] != df[prefix + '_answer']]
    if len(samples) == 0:
        return
    samples = samples.sample(count)
    for ts, fs, pr in zip(samples['text_step'], samples['formal_step'], samples[prefix + '_preds']):
        print('-' * 20)
        print('true step:')
        print(ts)
        print(fs)
        print()
        print('predict step:')
        print(pr)

Основная проблема заключается в зацикливание, например, "5 + 5 + 5 + 5 + ...", а параметр no_repeat_ngram_size не решает проблему с бесконечной цепью шагов. Похоже, что модель без пояснений не видит зависимостей

In [ ]:
show_true(test, 5, 'formal')
show_false(test, 5, 'formal')

In [ ]:
dataset = MathDataset(test, train, use_promt=arxiv_promt)
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 =

In [ ]:
preds = cot_inference(loader)

In [ ]:
test['arxiv_preds'] = preds
test['arxiv_answer'] = preds
test['arxiv_answer'] = test['arxiv_answer'].apply(extract_answer)

Результаты близки к тем, что описаны в статье для GPT3 с таким же количеством параметров, похоже, что BLOOM не потерял из-за многоязычности. Пропусков ответа нет, точность около 40-50%

In [ ]:
accuracy_score(test.processed_answer, test.arxiv_answer.fillna(''))

In [ ]:
show_true(test, 5, 'arxiv')
show_false(test, 5, 'arxiv')

Ответ совпадает с ходом мыслей - нет случайности в ответе, удивительно, что нет кейсов, когда после верных действий модель ошибается, т.е. неправильный ход мыслей и ответ идут с самого начала, будто непонятно условие.

Подтвердилась гипотеза, что модель справляется с дробями и прочими частями в среднем хуже, хотя набор действий тот же

Идея с формальными шагами провалилась, но хочется все равно подавать разные промты с надеждой угадать, поэтому осталось как в статье подавать текстовое описание, но в train выборке большая часть ответов длиннее максимального из промта авторов, поэтому оставим только короткие для выбора

In [ ]:
train['len_text_step'] = train.text_step.apply(lambda x: len(tokenizer.encode(x)))
train = train[train.len_text_step < 50].reset_index(drop=True)

In [ ]:
dataset = MathDataset(test, train, num_samples=8, step_col='text_step')
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

Q: John scores 2 shots worth 2 points and 1 shot worth 3 points every 4 minutes.  He plays for 2 periods.  Each period is 12 minutes.  How many points does he score?
A: He played for 2*12=24 minutes He gets 2*2=4 points per minute from the two-point shots So he scores 4+3=7 points per minute There are 24/4=6 intervals So he scores 7*6=42 points  The answer is 42
Q: Mona plays a multiplayer video game on the weekends. She joins groups with four other players to complete objectives in the game. The past weekend, she joined 9 groups. One of the groups included two players she had grouped with before that weekend, and another group included one person she had grouped with before. How many unique players did Mona group with on the video game that weekend?
A: Mona joined 9 groups, so she played with 9 * 4 = 36 people. She grouped with 2 + 1 = 3 people twice. Thus, she played with 36 - 3 = 33 unique players that weekend.  The answer is 33
Q: Michelle loves sugar. She bought 14 chocolate bars,

In [ ]:
preds = cot_inference(loader)

In [ ]:
test['text_preds'] = preds
test['text_answer'] = preds
test['text_answer'] = test['text_answer'].apply(extract_answer)

In [ ]:
accuracy_score(test.processed_answer, test.text_answer.fillna(''))

In [ ]:
show_true(test, 5, 'text')
show_false(test, 5, 'text')

Эффект и по метрикам и по ответам около полученного в статье, сложные задачи встречаются реже и рандом не попадает

In [ ]:
test

### Ensemble CoT

In [ ]:
arxiv_promt = '''Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8
'''

За основу возьмем предыдущий вариант датасета только для статичного промта

In [ ]:
dataset = MathDataset(test, train, use_promt=arxiv_promt)
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 =

В статье авторы меняли temperature, top_k, top_p для генераций, из-за ресурсов количество генераций пришлось взять минимальным, поэтому используем те параметры, что давали максимальную точность на малом количестве сэмплов

In [ ]:
def ensemble_cot_inference(loader):
    preds = []
    for promt, answer in tqdm(loader):
        inputs = tokenizer(promt, return_tensors="pt", padding='longest')["input_ids"].to(device)
        outputs = model.generate(inputs,
                                 max_new_tokens = 128, 
                                 eos_token_id=tokenizer.encode('\n')[0],
                                 do_sample=True,
                                 top_k=40,
                                 temperature=0.5,
                                 num_return_sequences=3,
                                 num_beams = 3
                                )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded = [[elem[len(p):] for elem in d] for p, d in zip(promt, decoded)]
        preds.extend(decoded)
    return preds

In [ ]:
def majority_answer(variants):
    return Counter(map(extract_answer, variants)).most_common()[0][0]

In [ ]:
preds = ensemble_cot_inference(loader)

In [ ]:
test['ensemble_preds'] = preds
test['ensemble_answers'] = test['ensemble_preds'].apply(majority_answer)

Идея ансамблирования давно показывает результаты лучше, здесь не исключение, точность повысилась на 5-10% (сходится со статьей на результатах с такими параметрами), примечательно, что на нескольких примерах больше голосов было за неправильный ответ, хотя предыдущий метод выдавал правильный, но это скорее обычное явление ансамблирования

In [ ]:
accuracy_score(test.processed_answer, test.ensemble_answers)

### Bonus

Если рандом не покрывает нужные случаи, а длинный промт неэффективен, можно было бы посмотреть в сторону ансамбля промтов дополнительно к предыдущей идее, тогда количество генераций это кол-во промтов * num_return_sequences, что невозможно при таком размере модели

Хочется решить две проблемы



1.   Сделать алгоритм подбора промта детерминированным
2.   Подбирать промт для каждой задачи так, чтобы их решения были близки, соответсвтенно покроем нечастые вопросы про проценты, доли, дроби

Идея: хранить для каждого вопроса из train его эмбеддинг и для нового вопроса из test искать n похожих вопросов, чтобы составить из них промт с текстовыми шагами.

Вопросы короткие с небольшой смысловой нагрузкой, к тому же нужно получать эмбеддинг быстро из-за и так медленного BLOOM, поэтому выберем лекгий берт, искать будем с помощью Faiss. В итоге метод такой же, как в предыдущих экспериментах, длину промта установили ранее, получили такую же задачу по вычислительной сложности, только промт подбирается не так обще



In [ ]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
searcher = faiss.IndexFlatL2(train.embedding.iloc[0].shape[0]) 
searcher.add(np.array(list(train.embedding)))

In [ ]:
class MathDataset(Dataset):
    def __init__(self, df, examples, use_promt=None, num_samples=1, step_col=''):
        self.df = df
        self.examples = examples
        self.num_samples = num_samples
        self.step_col = step_col
        self.use_promt = use_promt

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        if self.use_promt:
            promt = self.use_promt
        else:
            query = encoder.encode([sample.question])
            _, matched_indexes = searcher.search(query, self.num_samples)
            example = self.examples.iloc[matched_indexes[0]]
            promt = ''
            for q, a, s in zip(example.question, example.answer, example[self.step_col]):
                if self.step_col:
                    promt += 'Q: ' + q + '\n' + 'A: ' + s + ' The answer is ' + a + '\n'
                else:
                    promt += 'Q: ' + q + '\n' + 'A: The answer is ' + a + '\n'
        
        promt += 'Q: ' + sample.question + '\nA: '
        return promt, sample.answer

In [ ]:
dataset = MathDataset(test, train, num_samples=8, step_col='text_step')
loader = DataLoader(dataset, batch_size=1, shuffle=False)
print(dataset[0][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Q: Anne is drawing pictures. She has 12 markers and she notices that each one lasts for about 1.5 drawings. If she has already made 8 drawings, how many more can she make before she runs out of markers?
A: She can make 18 drawings in total because 12 x 1.5 = 18 She can make ten more because 18 - 8 = 10  The answer is 10
Q: Sara's sister has 4 small notebooks in her closet. Last summer she ordered 6 more notebooks and then lost 2. How many notebooks does Sara's sister have now?
A: Sara's sister has 4 + 6 = 10 notebooks. After she loses 2, she will have: 10 - 2 = 8 notebooks.  The answer is 8
Q: Madeline has $48 and her brother has half as much. How much do they have together?
A: Madeline's brother has $48/2 = $24. Together, they have $48 + $24 = $72.  The answer is 72
Q: Jennifer will be 30 years old in ten years. At that time, her sister Jordana will be three times as old Jennifer. How old is Jennifer's sister now?
A: When Jennifer is 30 years old in ten years, her sister Jordana will 

In [ ]:
preds = ensemble_cot_inference(loader)

In [ ]:
test['faiss_preds'] = preds
test['faiss_answers'] = test['faiss_preds'].apply(majority_answer)

In [ ]:
accuracy_score(test.processed_answer, test.faiss_answers)

BLOOM больше не поднимался, поэтому не получилось протестировать, но даже статистически точность не должна ухудшиться с предыдущим методом